In [ ]:
from cartoframes.__version__ import __version__
print(__version__)
# note: this is actually from `develop`: pip install -e git+https://github.com/cartodb/cartoframes.git@develop#egg=cartoframes

In [ ]:
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.client import SQLClient
from cartoframes.data import Dataset
from cartoframes.viz import Map, Layer
import pandas as pd
import geopandas as gpd

creds = Credentials('eschbacher', 'x')

set_default_credentials(creds)

station_points = 'capital_bikeshare_stations_points_arlington'
usage_data = 'capitalbikeshare_tripdata_201907'

sql_client = SQLClient(creds)

In [ ]:
Dataset(usage_data).download(limit=2)

In [ ]:
Dataset(station_points).download(limit=2)

In [ ]:
from cartoframes.viz.helpers import size_continuous_layer

In [ ]:
from cartoframes.viz.helpers import color_continuous_layer

### Visualize the data

In [ ]:
sql = f'''
SELECT
    _ends.num_bike_dropoffs,
    _starts.num_bike_pickups,
    abs(_ends.num_bike_dropoffs - _starts.num_bike_pickups) as diff,
    CASE WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups > 0 THEN 1
         WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups = 0 THEN 0
         ELSE -1 END as diff_sign,
    _ends.num_bike_dropoffs - _starts.num_bike_pickups as diff_relative,
    _starts.station_id,
    row_number() OVER () as cartodb_id,
    ST_X(_starts.the_geom) as longitude,
    ST_Y(_starts.the_geom) as latitude,
    _starts.the_geom,
    ST_Transform(_starts.the_geom, 3857) as the_geom_webmercator
FROM
    (SELECT
      count(u.*) as num_bike_dropoffs,
      u.end_station_number::int as station_id,
      s.the_geom,
      s.cartodb_id
    FROM {station_points} as s
    JOIN {usage_data} as u
    ON u.end_station_number::int = s.gisid::int
    GROUP BY 2, 3, 4) as _ends
JOIN
    (SELECT
      count(u.*) as num_bike_pickups,
      u.start_station_number::int as station_id,
      s.the_geom,
      s.cartodb_id
    FROM {usage_data} as u
    JOIN {station_points} as s
    ON u.start_station_number::int = s.gisid::int
    GROUP BY 2, 3, 4) as _starts
ON _ends.station_id = _starts.station_id
'''
Map([
    size_continuous_layer(Dataset(sql), 'num_bike_pickups'),
#     color_continuous_layer(Dataset(sql), 'diff_relative')
    size_continuous_layer(Dataset(sql), 'num_bike_dropoffs', color='purple')  
])

In [ ]:
pickup_dropoff_points = 'capital_bikeshare_july_2019_pickup_dropoff'
Dataset(sql).upload(table_name=pickup_dropoff_points)

In [ ]:
Map([
    size_continuous_layer(pickup_dropoff_points, 'num_bike_pickups'),
#     color_continuous_layer(Dataset(sql), 'diff_relative')
    size_continuous_layer(pickup_dropoff_points, 'num_bike_dropoffs', color='purple')  
])

In [ ]:
from cartoframes.viz import WidgetList
value = 'diff'
title = 'Drop Offs '
diff_map = Layer(
        sql,
        style={
            'point': {
                'width': 'ramp(linear(sqrt(${0}), sqrt(globalMin(${0})), sqrt(globalMax(${0}))), {1})'.format(
                    value, [2, 40]),
                'color': 'opacity(ramp(${0}, antique), 0.8)'.format(
                    'diff_sign', 'diff_sign'),
                'strokeColor': 'opacity(#222,ramp(linear(zoom(),0,18),[0,0.6]))',
            }
        },
        popup={
            'hover': [{
                'title': title,
                'value': f'$diff_sign * ${value}'
            }]
        },
        legend={
            'type': {
                'point': 'size-continuous-point',
                'line': 'size-continuous-line',
                'polygon': 'size-continuous-polygon'
            },
            'title': title,
            'description': '',
            'footer': ''
        }
    )
diff_map

### Animating pickups/dropoffs over time

In [ ]:
sql = f'''
SELECT
    _ends.num_bike_dropoffs,
    _starts.num_bike_pickups,
    abs(_ends.num_bike_dropoffs - _starts.num_bike_pickups) as diff,
    CASE WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups > 0 THEN 1
         WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups = 0 THEN 0
         ELSE -1 END as diff_sign,
    _ends.num_bike_dropoffs - _starts.num_bike_pickups as diff_relative,
    _starts.station_id,
    row_number() OVER () as cartodb_id,
    ST_X(_starts.the_geom) as longitude,
    ST_Y(_starts.the_geom) as latitude,
    _starts.the_geom,
    ST_Transform(_starts.the_geom, 3857) as the_geom_webmercator,
    _ends.day_of_month::numeric as day_of_month
FROM
    (SELECT
      count(u.*) as num_bike_dropoffs,
      u.end_station_number::int as station_id,
      s.the_geom,
      s.cartodb_id,
      EXTRACT(DAY FROM end_date) as day_of_month
    FROM {station_points} as s
    JOIN {usage_data} as u
    ON u.end_station_number::int = s.gisid::int
    GROUP BY 2, 3, 4, 5) as _ends
JOIN
    (SELECT
      count(u.*) as num_bike_pickups,
      u.start_station_number::int as station_id,
      s.the_geom,
      s.cartodb_id,
      EXTRACT(DAY FROM start_date) as day_of_month
    FROM {usage_data} as u
    JOIN {station_points} as s
    ON u.start_station_number::int = s.gisid::int
    GROUP BY 2, 3, 4, 5) as _starts
ON _ends.station_id = _starts.station_id and _ends.day_of_month = _starts.day_of_month
'''

size_continuous_layer(Dataset(sql), 'diff', animate='day_of_month')

In [ ]:
cols = set(Dataset(pickup_dropoff_points).download(limit=5).columns) - {'the_geom', 'cartodb_id', 'the_geom_webmercator'}
sql = f'''
CREATE TABLE {pickup_dropoff_points}_isochrone AS
SELECT ST_MakeValid((cdb_isochrone(the_geom, 'walk', Array[600])).the_geom) as the_geom, cartodb_id,
    {','.join(cols)}
FROM {pickup_dropoff_points};
SELECT CDB_Cartodbfytable('eschbacher', '{pickup_dropoff_points}_isochrone');
'''

sql_client.execute(sql)

In [ ]:
Layer(f'{pickup_dropoff_points}_isochrone')

In [ ]:
from cartoframes.client import DataObsClient
do = DataObsClient(creds)

In [ ]:
do_aug = do.augment(
    f'{pickup_dropoff_points}_isochrone',
    [{"numer_id": "us.census.acs.B23006001",
         "denom_id": "us.census.acs.B01003001",
         "normalization": "denominated"}]
)

In [ ]:
do_aug.dataframe.head()

In [ ]:
do_aug.upload(table_name=f'{pickup_dropoff_points}_isochrone_augmented')

In [ ]:
color_continuous_layer(f'{pickup_dropoff_points}_isochrone_augmented', 'pop_25_64_2011_2015_by_total_pop')

### Run a Model

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipeline.fit(do_aug.dataframe[['num_bike_pickups', 'num_bike_dropoffs', 'pop_25_64_2011_2015_by_total_pop']])

### Add results to dataset

In [ ]:
do_aug.dataframe['labels'] = pipeline.steps[1][1].labels_.astype(str)

In [ ]:
do_aug.dataframe.head()

In [ ]:
do_aug.upload(table_name='demo_augmentation', if_exists='replace')

In [ ]:
results_map = Map(
    color_category_layer('demo_augmentation', 'labels', widget=True, palette='prism'),
    
)
results_map

In [ ]:
results_map.publish('Station Encoding')

### Map Grid

In [ ]:
from cartoframes.viz import MapGrid
results_map = Map(
    color_category_layer('demo_augmentation', 'labels')
)
MapGrid([
    results_map,
    Map(diff_map)
], 2, 1, viewport={'zoom': 11})